In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
ROOT_PATH = Path().resolve().parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(ROOT_PATH, 1)

In [3]:
import ipywidgets as widgets
from IPython.display import display
from autoeq.constants import PEQ_CONFIGS
from autoeq.batch_processing import batch_processing
ROOT_PATH = Path().resolve().parent
from dbtools.rtings_crawler import RtingsCrawler
from dbtools.crinacle_crawler import CrinacleCrawler
from dbtools.oratory1990_crawler import Oratory1990Crawler
from dbtools.innerfidelity_crawler import InnerfidelityCrawler
from dbtools.headphonecom_crawler import HeadphonecomCrawler
from dbtools.prune_results import prune_results
from dbtools.update_result_indexes import update_all_indexes
from dbtools.create_webapp_data import write_entries_and_measurements, write_targets
from dbtools.constants import TARGETS_PATH, MEASUREMENTS_PATH, RESULTS_PATH

## Crawling and Parsing
Additional Python packages are required for processing the measurements:
```bash
python -m pip install -U -r dbtools/requirements.txt
```

This notebook uses IPyWidgets
```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
```

Finally install IPython kernel
```bash
python -m ipykernel install --user --name="autoeq"
```

Measurement crawlers require [Google Chrome](https://www.google.com/chrome/) installed and
[ChromeDriver](https://googlechromelabs.github.io/chrome-for-testing/) binary in the measurements folder (or anywhere
in the PATH).

Measurement crawlers also require C++. This should be installed by default on Linux but on Windows you need to install
Microsoft Visual Studio build tools for this. https://visualstudio.microsoft.com/downloads/ ->
"Tools for Visual Studio 2019" -> "Build Tools for Visual Studio 2019".

oratory1990 crawler requires Ghostscript installed: https://www.ghostscript.com/download/gsdnld.html

### Crinacle
Download measurement data from Drive folder to `measurements/crinacle/raw_data/` before running this!

* `IEM Measurements/IEC60318-4 IEM Measurements (TSV txt)` into `AutoEq/measurements/crinacle/raw_data/IEC60318-4 IEM Measurements (TSV txt)`
* `IEM Measurements/4620 IEM Measurements` into `AutoEq/measurements/crinacle/raw_data/4620 IEM Measurements`
* `HP Measurements/EARS + 711 (TSV txt) (Legacy)` into `AutoEq/measurements/crinacle/raw_data/EARS + 711 (TSV txt) (Legacy)`
* `GRAS 43AG-7` into `AutoEq/measurements/crinacle/raw_data/GRAS 43AG-7`

In [ ]:
crawler = CrinacleCrawler()
crawler.run()
display(crawler.widget)

In [14]:
crawler.prune_measurements(dry_run=True)

No measurements need to be pruned


In [ ]:
crawler.process(new_only=True)

### oratory1990
oratory1990 crawler fetches all measurements from https://www.reddit.com/r/oratory1990/wiki/index/list_of_presets/, downloads PDFs and reads the frequency response measurement data from the PDFs. Parsing the PDFs requires [Ghostscript](https://www.ghostscript.com/download/gsdnld.html) to be installed on the system.

In [34]:
crawler = Oratory1990Crawler()

In [ ]:
crawler.run()
display(crawler.widget)

In [36]:
crawler.prune_measurements(dry_run=False)

Removed "C:\Users\jaakko\code\AutoEq\measurements\oratory1990\data\over-ear\AKG K712 (Dekoni sheepskin Earpads).csv"


In [41]:
crawler.process(new_only=True)

  0%|          | 0/612 [00:00<?, ?it/s]

### Rtings
Rtings crawler fetches all measurements from https://www.rtings.com/headphones/1-[2,4,5]/graph and downloads raw FR JSON files and parses them.

In [44]:
crawler = RtingsCrawler()

In [ ]:
crawler.run()
display(crawler.widget)

In [45]:
crawler.prune_measurements(dry_run=True)

No measurements need to be pruned


In [53]:
crawler.process(new_only=True)

  0%|          | 0/723 [00:00<?, ?it/s]

## Rename Measurements
Sometimes measurements are named incorrectly or previously only one sample existed and now multiple samples have been measured and so the original one needs to be renamed as "<name> (sample 1)"

In [ ]:
crinacle = CrinacleCrawler()
headphonecom = HeadphonecomCrawler()
innerfidelity = InnerfidelityCrawler()
oratory1990 = Oratory1990Crawler()
rtings = RtingsCrawler(driver=oratory1990.driver)

renames = [
    {'old_name': 'Audeze LCD-2 (Rev. 2)', 'new_name': 'Audeze LCD-2 (Rev 2)', 'crawlers': [headphonecom]},
]
for rename in renames:
    for crawler in rename['crawlers']:
        crawler.rename_measurement(old_name=rename['old_name'], new_name=rename['new_name'], dry_run=True)

## Prune Results
Check if obsolete results (e.g. because of renaming) exist and remove them

In [60]:
prune_results(databases=['crinacle', 'oratory1990', 'Headphone.com Legacy', 'Innerfidelity', 'Rtings'], dry_run=True)

C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\1MORE MK801.csv
Removed "crinacle\GRAS 43AG-7 over-ear\1MORE MK801"
C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\Abyss AB-1266 Phi.csv
Removed "crinacle\GRAS 43AG-7 over-ear\Abyss AB-1266 Phi"
C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\Abyss AB-1266 Phi CC.csv
Removed "crinacle\GRAS 43AG-7 over-ear\Abyss AB-1266 Phi CC"
C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\Abyss Diana Phi.csv
Removed "crinacle\GRAS 43AG-7 over-ear\Abyss Diana Phi"
C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\Abyss Diana V2.csv
Removed "crinacle\GRAS 43AG-7 over-ear\Abyss Diana V2"
C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\Acoustic Research AR-H1.csv
Removed "crinacle\GRAS 43AG-7 over-ear\Acoustic Research AR-H1"
C:\Users\jaakko\code\AutoEq\measurements\crinacle\data\in-ear\GRAS 43AG-7\AKG K2

## Update Results
Creates new results from the measurements. `eq_kwargs` are parameters shared by all jobs.

In [4]:
eq_kwargs = {
    'parametric_eq': True, 'ten_band_eq': True, 'convolution_eq': True,
    'parametric_eq_config': [PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF'], PEQ_CONFIGS['4_PEAKING_WITH_HIGH_SHELF']],
    'fs': [44100, 48000],
    'thread_count': 0,
}

#### oratory1990 Over-ear

In [42]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('oratory1990', 'data', 'over-ear'),
    output_dir=RESULTS_PATH.joinpath('oratory1990', 'over-ear'),
    target=TARGETS_PATH.joinpath('Harman over-ear 2018 without bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/1 [00:00<?, ?it/s]

#### oratory1990 In-ear

In [7]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('oratory1990', 'data', 'in-ear'),
    output_dir=RESULTS_PATH.joinpath('oratory1990', 'in-ear'),
    target=TARGETS_PATH.joinpath('AutoEq in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/188 [00:00<?, ?it/s]

#### oratory1990 Earbud

In [8]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('oratory1990', 'data', 'earbud'),
    output_dir=RESULTS_PATH.joinpath('oratory1990', 'earbud'),
    target=TARGETS_PATH.joinpath('AutoEq in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/1 [00:00<?, ?it/s]

#### crinacle GRAS 43AG-7 On-ear

In [9]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('crinacle', 'data', 'over-ear', 'GRAS 43AG-7'),
    output_dir=RESULTS_PATH.joinpath('crinacle', 'GRAS 43AG-7 over-ear'),
    target=TARGETS_PATH.joinpath('Harman over-ear 2018 without bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/331 [00:00<?, ?it/s]

#### crinacle EARS+711 Over-ear

In [10]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('crinacle', 'data', 'over-ear', 'EARS + 711'),
    output_dir=RESULTS_PATH.joinpath('crinacle', 'EARS + 711 over-ear'),
    target=TARGETS_PATH.joinpath('crinacle EARS + 711 Harman over-ear 2018 without bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/65 [00:00<?, ?it/s]

#### crinacle 4620 In-ear

In [11]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('crinacle', 'data', 'in-ear', 'Bruel & Kjaer 4620'),
    output_dir=RESULTS_PATH.joinpath('crinacle', 'Bruel & Kjaer 4620 in-ear'),
    target=TARGETS_PATH.joinpath('Diffuse field 5128 -1dB per octave.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/169 [00:00<?, ?it/s]

#### crinacle 711 In-ear

In [13]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('crinacle', 'data', 'in-ear', '711'),
    output_dir=RESULTS_PATH.joinpath('crinacle', '711 in-ear'),
    target=TARGETS_PATH.joinpath('AutoEq in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/1437 [00:00<?, ?it/s]

#### Rtings Over-ear

In [54]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Rtings', 'data', 'over-ear'),
    output_dir=RESULTS_PATH.joinpath('Rtings', 'over-ear'),
    target=TARGETS_PATH.joinpath('Rtings Harman over-ear 2018 without bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/6 [00:00<?, ?it/s]

#### Rtings In-ear

In [49]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Rtings', 'data', 'in-ear'),
    output_dir=RESULTS_PATH.joinpath('Rtings', 'in-ear'),
    target=TARGETS_PATH.joinpath('Rtings AutoEq in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/15 [00:00<?, ?it/s]

#### Rtings Earbud

In [17]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Rtings', 'data', 'earbud'),
    output_dir=RESULTS_PATH.joinpath('Rtings', 'earbud'),
    target=TARGETS_PATH.joinpath('Rtings Autoeq in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/23 [00:00<?, ?it/s]

#### Innerfidelity In-ear

In [18]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Innerfidelity', 'data', 'over-ear'),
    output_dir=RESULTS_PATH.joinpath('Innerfidelity', 'over-ear'),
    target=TARGETS_PATH.joinpath('Innerfidelity Harman over-ear 2018 without bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/612 [00:00<?, ?it/s]

#### Innerfidelity In-ear

In [19]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Innerfidelity', 'data', 'in-ear'),
    output_dir=RESULTS_PATH.joinpath('Innerfidelity', 'in-ear'),
    target=TARGETS_PATH.joinpath('Innerfidelity AutoEq in-ear.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/307 [00:00<?, ?it/s]

#### Innerfidelity Earbud

In [20]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Innerfidelity', 'data', 'earbud'),
    output_dir=RESULTS_PATH.joinpath('Innerfidelity', 'earbud'),
    target=TARGETS_PATH.joinpath('Innerfidelity AutoEq in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/15 [00:00<?, ?it/s]

#### Headphone.com Legacy Over-ear

In [21]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Headphone.com Legacy', 'data', 'over-ear'),
    output_dir=RESULTS_PATH.joinpath('Headphone.com Legacy', 'over-ear'),
    target=TARGETS_PATH.joinpath('Headphone.com Legacy Harman over-ear 2018 without bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/225 [00:00<?, ?it/s]

#### Headphone.com Legacy In-ear

In [22]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Headphone.com Legacy', 'data', 'in-ear'),
    output_dir=RESULTS_PATH.joinpath('Headphone.com Legacy', 'in-ear'),
    target=TARGETS_PATH.joinpath('Headphone.com Legacy AutoEq in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/94 [00:00<?, ?it/s]

#### Headphone.com Legacy Earbud

In [23]:
_ = batch_processing(
    input_dir=MEASUREMENTS_PATH.joinpath('Headphone.com Legacy', 'data', 'earbud'),
    output_dir=RESULTS_PATH.joinpath('Headphone.com Legacy', 'earbud'),
    target=TARGETS_PATH.joinpath('Headphone.com Legacy AutoEq in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

## Update Indexes
Updates recommended results, full results, DB specific results, HeSuVi results and ranking table.

In [26]:
update_all_indexes()

Creating ranking index...


  0%|          | 0/3665 [00:00<?, ?it/s]

Creating recommendations index...
Creating full index...
Creating source indices...
Creating HeSuVi ZIP archive...


  0%|          | 0/3665 [00:00<?, ?it/s]

C:\Users\jaakko\AppData\Local\Programs\Python\Python311\Lib\zipfile.py:1550: UserWarning: Duplicate name: 'eq/KBEAR/Neon.txt'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


#### Data for webapp

In [57]:
write_entries_and_measurements()

  0%|          | 0/4599 [00:00<?, ?it/s]

In [27]:
write_targets()

## Deploy
1. Add files to Git, commit and push
2. Upload webapp data to server

# Sandbox
Don't run these! Random exploration while developing.

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from pathlib import Path
from tqdm.auto import tqdm
import re
import requests
from selenium.webdriver.common.by import By
import json
from bs4 import BeautifulSoup
import numpy as np
import json
from autoeq.frequency_response import FrequencyResponse

In [32]:
crawler = CrinacleCrawler()

In [34]:
crawler = Oratory1990Crawler(redownload=False)

In [ ]:
crawler = RtingsCrawler()

In [86]:
version = '1-2'
html = requests.get(f'https://www.rtings.com/headphones/{version}/graph').text
document = BeautifulSoup(html, 'html.parser')

In [87]:
test_bench = json.loads(document.find(class_='graph_tool_page').get('data-props'))['test_bench']

In [88]:
list(test_bench.keys())

['name', 'id', 'comparable_products', 'tests']

In [89]:
known_test_ids = {test['id']: {'name': test['name'], 'equivalent': test['equivalent_test_ids']} for test in test_bench['tests']}
for key, val in known_test_ids.items():
    print(key, val)

2037 {'name': 'Bass', 'equivalent': ['436', '996', '1186', '2037', '3163']}
2044 {'name': 'Mid', 'equivalent': ['437', '1007', '1197', '2044', '3170']}
2050 {'name': 'Treble', 'equivalent': ['438', '1013', '1203', '2050', '3176']}
2056 {'name': 'Consistency L', 'equivalent': ['569', '1019', '1209', '2056', '3185', '4007', '7913', '21560']}
2057 {'name': 'Consistency R', 'equivalent': ['570', '1020', '1210', '2057', '3186', '4008', '7914', '21561']}
2060 {'name': 'Raw FR L', 'equivalent': ['1344', '2060', '3182']}
2061 {'name': 'Raw FR R', 'equivalent': ['1343', '2061', '3183']}
2069 {'name': 'PRTF', 'equivalent': ['1965', '2069', '3196', '4021', '7950', '21598']}
2074 {'name': 'Group Delay', 'equivalent': ['1621', '2074', '3189', '4014', '7943', '21590']}
2075 {'name': 'Phase Response', 'equivalent': ['529', '1030', '1220', '2075', '3190', '4015']}
2085 {'name': 'Distortion', 'equivalent': ['329', '1037', '1227', '2085']}
2090 {'name': 'Noise Isolation', 'equivalent': ['349', '1054', '

In [90]:
unknown_test_ids = {}
for product in test_bench['comparable_products']:
    test_ids = [test_result['test']['original_id'] for test_result in product['review']['test_results']]
    for test_id in test_ids:
        if test_id not in known_test_ids:
            unknown_test_ids[test_id] = {'name': product['fullname'], 'id': product['id']}
unknown_test_ids

{'329': {'name': 'AKG K391-NC', 'id': '234'},
 '349': {'name': 'AKG K391-NC', 'id': '234'},
 '353': {'name': 'AKG K391-NC', 'id': '234'},
 '436': {'name': 'AKG K391-NC', 'id': '234'},
 '437': {'name': 'AKG K391-NC', 'id': '234'},
 '438': {'name': 'AKG K391-NC', 'id': '234'},
 '529': {'name': 'AKG K391-NC', 'id': '234'},
 '569': {'name': 'AKG K391-NC', 'id': '234'},
 '570': {'name': 'AKG K391-NC', 'id': '234'}}

In [91]:
for test_id, product in unknown_test_ids.items():
    res = requests.post('https://www.rtings.com/api/v2/safe/graph_tool__product_graph_data_url', data={
        'named_version': 'public',
        'product_id': product['id'],
        'test_original_id': test_id,
    })
    print(f'{test_id}: {res.json()}')

329: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/mcuBmpTu/graph-distortion.json'}]}}}}
349: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/l2VvLSvF/graph-isolation.json'}]}}}}
353: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/pPwMCFvC/graph-leakage.json'}]}}}}
436: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/liYFzpsm/graph-bass.json'}]}}}}
437: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/KhmOchbD/graph-mid.json'}]}}}}
438: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/wzSsQqzQ/graph-treble.json'}]}}}}
529: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/WsFGD2nr/graph-phase-response.json'}]}}}}
569: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/LK19W26h/graph-